## Centralized Learning


In [0]:
# For Privacy 
import syft as sy
# For Machine Learning 
from torch import nn, optim

# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much the missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data[0])

In [2]:
#train the model! 
train() 

 
0.7935236096382141
0.21217259764671326
0.12388776242733002
0.08632121235132217
0.06183892861008644
0.044710226356983185
0.0325467512011528
0.02383730188012123
0.017555570229887962
0.012994087301194668
0.009660916402935982
0.007211238145828247
0.005401470698416233
0.004058181773871183
0.003056995337828994
0.002308045281097293
0.0017459806986153126
0.0013229941250756383
0.0010039110202342272
0.0007627044105902314


## Federated Learning (multiple data sources)

In [0]:
# NEW - create 2 different users

bob = sy.VirtualWorker(id="jack")
alice = sy.VirtualWorker(id="jill")

# get pointers to training data on each worker by
# sending some training data to jack and jill
data_jack = data[0:2].send(jack)
target_jack = target[0:2].send(jack)

data_jill = data[2:].send(jill)
target_jill = target[2:].send(jill)

# organize pointers into a list
datasets = [(data_jack,target_jack),(data_jill,target_jill)]

# Initalize A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much the missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # NEW) get model (with gradients)
            model.get()

            # 5) change those weights
            opt.step()

            # 6) print our progress
            print(loss.get().data[0]) # NEW) slight edit... need to call .get() on loss

In [3]:
train()



0.49059900641441345
0.48288848996162415
0.5257140398025513
0.20319302380084991
0.3727419972419739
0.14014576375484467
0.2604595422744751
0.10102146863937378
0.18343526124954224
0.07364650815725327
0.13027068972587585
0.05409587174654007
0.0932079330086708
0.039966512471437454
0.06712788343429565
0.029659217223525047
0.04862105846405029
0.022084159776568413
0.03538985550403595


## Secure Multi Party Computation 

In [4]:
# We can Encrypt with SMPC!

Q = 1234567891011

x = 25

import random

def encrypt(x):
    share_a = random.randint(0,Q)
    share_b = random.randint(0,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)
  
  
encrypt(x)

508598083450, 1210525625153, 750012073444


In [5]:
# We can Decrypt with SMPC! 
def decrypt(*shares):
    return sum(shares) % Q
  
  
a,b,c = encrypt(25)

decrypt(a, b, c)

25


In [6]:
# We can even perform operations on encrypted data with SMPC!
x = encrypt(25)
y = encrypt(5)

def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z
  
decrypt(*add(x,y))

30


# Secure Multi Party Computation on an encrypted neural network + encrypted data

In [7]:
#for privacy 
import syft as sy
#for diabetes image data 
import my_dataset

# Step 1 - Create 3 data sources
me = local_worker
bob = sy.VirtualWorker(id="bob", is_client_worker=False)
alice = sy.VirtualWorker(id="alice", is_client_worker=False)

# Step 2 - import our imagedataset
data, target = my_dataset.import()

# Step 3 - Create our neural network
model = sy.zeros(2,1)

# Step 4 - encrypt the model and encrypt the data
data = data.fix_precision().share(alice, bob)
target = target.fix_precision().share(alice, bob)
model = model.fix_precision().share(alice, bob)

# Step 5 - train!
for i in range(10):
    pred = data.mm(model)
    grad = pred - target
    update = data.transpose(0,1).mm(grad)

    model = model - update * 0.1
    loss = grad.get().decode().abs().sum()
    print(loss)

 
2.0
1.6000000014901161
1.2999999821186066
1.0720000118017197
0.896000012755394
0.7599999904632568
0.6499999910593033
0.5619999840855598
0.4880000092089176
0.42799999564886093

